In [1]:
############################################################
################### CREATE HISTORIC DATA ###################
############################################################
## Tool for reading the table and removing the last rows  ##
## one by one to create historical data.                  ##
############################################################
####################### INSTRUCTIONS #######################
### To generate historical data:
### 1) get data from www.football-data.co.uk and ensure that there are necessary columns
### 2) rename the file downloaded with the results of matches to E0.csv
### 3) tune argument of function calc_features() to create dataset with variables (max possible )
### 4) run this script
### 5) In the output file 'hist_data':
### 5.1) remove the column with zeros
### 5.2) add columns with the historical odds in the order [h_course, d_course, a_course]




import yaml
import pandas as pd

with open("config.yaml", 'r') as configuration:
    config = yaml.safe_load(configuration)


def calc_features(n):
    def read_data(n):
        # n=0 - get all rows
        # n=1 - remove 1 row
        # n=k - remove k row
        results = pd.read_csv('E0.csv', parse_dates=['Date'], dayfirst=True)
        HOME_TEAM = results.iloc[len(results)-1-n].HomeTeam
        AWAY_TEAM = results.iloc[len(results)-1-n].AwayTeam
        match_res = results.iloc[[len(results)-1-n],:][['Date','HomeTeam','AwayTeam','FTR']]
        match_res.index = [0]
        results = results.drop(results.tail(n+1).index)
        return HOME_TEAM, AWAY_TEAM, match_res, results

    HOME_TEAM, AWAY_TEAM, match_res, results = read_data(0)
    results = results[['Date','HomeTeam','AwayTeam','FTHG','FTAG','FTR','HS','AS','HST','AST','HC','AC','HY','AY','HR','AR']]
    table = pd.DataFrame(set(results.HomeTeam), columns = ['Team'])
    results2 = results.copy()
    results2 = results2.drop(['HomeTeam'], axis=1)
    results2['HomeTeam'] = results2.AwayTeam
    results2 = results2.drop(['AwayTeam'], axis=1)
    results2['HoA'] = 'A'

    results['HoA'] = 'H'
    results = results.drop(['AwayTeam'], axis=1)
    results

    results = pd.concat([results, results2], axis=0,ignore_index=True)
    results.rename(columns={'HomeTeam':'Team'}, inplace=True)
    #results['Date'] = pd.to_datetime(results['Date'], format='%d/%m/%y')
    results = results.sort_values(by=['Date'], ascending=False)

    def punkty_zdobyte(results):
        if results.FTR == 'D':
            return 1
        if results.FTR == results.HoA:
            return 3
        if results.FTR != results.HoA:
            return 0

    def gole_zdobyte(results):
        if results.HoA == 'A':
            return results.FTAG
        if results.HoA == 'H':
            return results.FTHG

    def gole_stracone(results):
        if results.HoA == 'A':
            return results.FTHG
        if results.HoA == 'H':
            return results.FTAG

    def strzaly_oddane(results):
        if results.HoA == 'A':
            return results.AS
        if results.HoA == 'H':
            return results.HS

    def strzaly_otrzymane(results):
        if results.HoA == 'A':
            return results.HS
        if results.HoA == 'H':
            return results.AS

    def strz_cel_oddane(results):
        if results.HoA == 'A':
            return results.AST
        if results.HoA == 'H':
            return results.HST

    def strz_cel_otrzymane(results):
        if results.HoA == 'A':
            return results.HST
        if results.HoA == 'H':
            return results.AST

    def kornery_wykonane(results):
        if results.HoA == 'A':
            return results.AC
        if results.HoA == 'H':
            return results.HC

    def kornery_bronione(results):
        if results.HoA == 'A':
            return results.HC
        if results.HoA == 'H':
            return results.AC
    def otrzymane_zolte_kartki(results):
        if results.HoA == 'A':
            return results.AY
        if results.HoA == 'H':
            return results.HY

    def otrzymane_czerwone_kartki(results):
        if results.HoA == 'A':
            return results.AR
        if results.HoA == 'H':
            return results.HR


    results['pts'] = results.apply(lambda x: punkty_zdobyte(x), axis=1)
    results['goal_zdob'] = results.apply(lambda x: gole_zdobyte(x), axis=1)
    results['goal_strc'] = results.apply(lambda x: gole_stracone(x), axis=1)
    results['sh_odd'] = results.apply(lambda x: strzaly_oddane(x), axis=1)
    results['sh_otrz'] = results.apply(lambda x: strzaly_otrzymane(x), axis=1)
    results['sot_odd'] = results.apply(lambda x: strz_cel_oddane(x), axis=1)
    results['sot_otrz'] = results.apply(lambda x: strz_cel_otrzymane(x), axis=1)
    results['cor_wyk'] = results.apply(lambda x: kornery_wykonane(x), axis=1)
    results['cor_bro'] = results.apply(lambda x: kornery_bronione(x), axis=1)
    results['yel_card'] = results.apply(lambda x: otrzymane_zolte_kartki(x), axis=1)
    results['red_card'] = results.apply(lambda x: otrzymane_czerwone_kartki(x), axis=1)

    # Preparing data:
    # - group by HomeTeam
    # - we are reversing the datasets to take into consideration the latest matches

    def split(data, f):
        grouped = data.groupby(f)
        return [g for _, g in grouped], list(grouped.groups.keys())

    results_split, results_split_names = split(results, 'Team')
    # We take the form of the team - different versions are possible here

    # Lista zdobytych punktów w ostatnich n meczach
    def team_form_pts_mean(team, n):
        return team.pts[:n].mean()

    # Średnia zdobytych bramek w ostatnich n meczach
    def team_form_goal_mean(team, n):
        return team.goal_zdob[:n].mean()

    # Średnia straconych bramek w ostatnich n meczach
    def team_form_goal_strac_mean(team, n):
        return team.goal_strc[:n].mean()

    # Średnia strzałów oddanych w ostatnich n meczach
    def team_form_shot_odd_mean(team, n):
        return team.sh_odd[:n].mean()

    # Średnia strzałów przyjetych w ostatnich n meczach
    def team_form_shot_otrz_mean(team, n):
        return team.sh_otrz[:n].mean()

    # Średnia strzałów celnych oddanych w ostatnich n meczach
    def team_form_shot_trg_odd_mean(team, n):
        return team.sot_odd[:n].mean()

    # Średnia strzałów celnych przyjetych w ostatnich n meczach
    def team_form_shot_trg_otrz_mean(team, n):
        return team.sot_otrz[:n].mean()

    # Średnia kornerów wykonanych w ostatnich n meczach
    def team_form_cor_wyk_mean(team, n):
        return team.cor_wyk[:n].mean()

    # Średnia kornerów bronionych w ostatnich n meczach
    def team_form_kor_bro_mean(team, n):
        return team.cor_bro[:n].mean()

    # Średnia liczba zółtych kartek w ostatnich n meczach
    def team_form_yel_card_mean(team, n):
        return team.yel_card[:n].mean()

    # Średnia liczba czerwonych kartek w ostatnich n meczach
    def team_form_red_card_mean(team, n):
        return team.red_card[:n].mean()

    # Tworzymy zmiene oparte na liczbie zdobytych punktów w ostatnich n meczach
    form_var = pd.concat([pd.DataFrame(results_split_names),
                          # Zmiene oparte na liczbie zdobytych punktów w ostatnich n meczach
                           pd.DataFrame(list(map(lambda x: team_form_pts_mean(x, 3), results_split))),
                           pd.DataFrame(list(map(lambda x: team_form_pts_mean(x, 5), results_split))),
                           pd.DataFrame(list(map(lambda x: team_form_pts_mean(x, 7), results_split))),
                          # Zmiene oparte na liczbie zdobytych bramek w ostatnich n meczach
                          pd.DataFrame(list(map(lambda x: team_form_goal_mean(x, 3), results_split))),
                          pd.DataFrame(list(map(lambda x: team_form_goal_mean(x, 5), results_split))),
                          pd.DataFrame(list(map(lambda x: team_form_goal_mean(x, 7), results_split))),
                          # Zmiene oparte na liczbie straconych bramek w ostatnich n meczach
                          pd.DataFrame(list(map(lambda x: team_form_goal_strac_mean(x, 3), results_split))),
                          pd.DataFrame(list(map(lambda x: team_form_goal_strac_mean(x, 5), results_split))),
                          pd.DataFrame(list(map(lambda x: team_form_goal_strac_mean(x, 7), results_split))),
                          # Zmiene oparte na liczbie oddanych strzałów w ostatnich n meczach
                          pd.DataFrame(list(map(lambda x: team_form_shot_odd_mean(x, 3), results_split))),
                          pd.DataFrame(list(map(lambda x: team_form_shot_odd_mean(x, 5), results_split))),
                          pd.DataFrame(list(map(lambda x: team_form_shot_odd_mean(x, 7), results_split))),
                          # Zmiene oparte na liczbie otrzymanych strzałów w ostatnich n meczach
                          pd.DataFrame(list(map(lambda x: team_form_shot_otrz_mean(x, 3), results_split))),
                          pd.DataFrame(list(map(lambda x: team_form_shot_otrz_mean(x, 5), results_split))),
                          pd.DataFrame(list(map(lambda x: team_form_shot_otrz_mean(x, 7), results_split))),

                          # Zmiene oparte na liczbie oddanych strzałów w ostatnich n meczach
                          pd.DataFrame(list(map(lambda x: team_form_shot_trg_odd_mean(x, 3), results_split))),
                          pd.DataFrame(list(map(lambda x: team_form_shot_trg_odd_mean(x, 5), results_split))),
                          pd.DataFrame(list(map(lambda x: team_form_shot_trg_odd_mean(x, 7), results_split))),
                          # Zmiene oparte na liczbie otrzymanych strzałów w ostatnich n meczach
                          pd.DataFrame(list(map(lambda x: team_form_shot_trg_otrz_mean(x, 3), results_split))),
                          pd.DataFrame(list(map(lambda x: team_form_shot_trg_otrz_mean(x, 5), results_split))),
                          pd.DataFrame(list(map(lambda x: team_form_shot_trg_otrz_mean(x, 7), results_split))),

                          # Zmiene oparte na liczbie kornerów wykonanych w ostatnich n meczach
                          pd.DataFrame(list(map(lambda x: team_form_cor_wyk_mean(x, 3), results_split))),
                          pd.DataFrame(list(map(lambda x: team_form_cor_wyk_mean(x, 5), results_split))),
                          pd.DataFrame(list(map(lambda x: team_form_cor_wyk_mean(x, 7), results_split))),
                          # Zmiene oparte na liczbie korneró bronionych w ostatnich n meczach
                          pd.DataFrame(list(map(lambda x: team_form_kor_bro_mean(x, 3), results_split))),
                          pd.DataFrame(list(map(lambda x: team_form_kor_bro_mean(x, 5), results_split))),
                          pd.DataFrame(list(map(lambda x: team_form_kor_bro_mean(x, 7), results_split))),
                          # Zmiene oparte na liczbie zółtych kartek w ostatnich n meczach
                          pd.DataFrame(list(map(lambda x: team_form_yel_card_mean(x, 3), results_split))),
                          pd.DataFrame(list(map(lambda x: team_form_yel_card_mean(x, 5), results_split))),
                          pd.DataFrame(list(map(lambda x: team_form_yel_card_mean(x, 7), results_split))),
                          # Zmiene oparte na liczbie czerwonych kartek w ostatnich n meczach
                          pd.DataFrame(list(map(lambda x: team_form_red_card_mean(x, 3), results_split))),
                          pd.DataFrame(list(map(lambda x: team_form_red_card_mean(x, 5), results_split))),
                          pd.DataFrame(list(map(lambda x: team_form_red_card_mean(x, 7), results_split))),
                          ],axis=1,ignore_index=True)

    # Nazywamy zmienne
    form_var.columns = ['Team',
                        'pts_3','pts_5','pts_7',
                        'gz_3','gz_5','gz_7',
                        'gs_3','gs_5','gs_7',
                        'sh_od_3','sh_od_5','sh_od_7',
                        'sh_ot_3','sh_ot_5','sh_ot_7',
                        'sot_od_3','sot_od_5','sot_od_7',
                        'sot_ot_3','sot_ot_5','sot_ot_7',
                        'cw_3','cw_5','cw_7',
                        'cb_3','cb_5','cb_7',
                        'yc_3','yc_5','yc_7',
                        'rc_3','rc_5','rc_7',
                       ]

    form_var = form_var.set_index('Team')

    ###################################################################
    # Create a league table and other variables (not based on time)
    ###################################################################

    HOME_TEAM, AWAY_TEAM, match_res, results = read_data(n)
    results = results[['Date','HomeTeam','AwayTeam','FTHG','FTAG','FTR','HS','AS','HST','AST','HC','AC']]
    #table = pd.DataFrame(set(results.HomeTeam), columns = ['Team'])

    def home_points(results):
        if results.FTR == 'H':
            return 3
        if results.FTR == 'A':
            return 0
        if results.FTR == 'D':
            return 1

    def away_points(results):
        if results.FTR == 'H':
            return 0
        if results.FTR == 'A':
            return 3
        if results.FTR == 'D':
            return 1

    results['H_pts'] = results.apply(lambda x: home_points(x), axis=1)
    results['A_pts'] = results.apply(lambda x: away_points(x), axis=1)

    home_table = results.groupby('HomeTeam').sum()[['FTHG','FTAG','HS','AS','HST','AST','HC','AC','H_pts']]
    away_table = results.groupby('AwayTeam').sum()[['FTHG','FTAG','HS','AS','HST','AST','HC','AC','A_pts']]
    home_table.columns = ['H_goal_zdob','H_goal_strc','H_strzaly_oddane','H_strzaly_dopuszczone','H_strz_cel_oddane','H_strz_cel_dopuszczone','H_kornery_wyk','H_kornery_bro','H_pts']
    away_table.columns = ['A_goal_strc','A_goal_zdob','A_strzaly_dopuszczone','A_strzaly_oddane','A_strz_cel_oddane','A_strz_cel_dopuszczone','A_kornery_wyk','A_kornery_bro','A_pts']

    res_table = pd.concat([home_table, away_table], axis=1)

    res_table['goal_zdob'] = res_table.H_goal_zdob + res_table.A_goal_zdob
    res_table['goal_strc'] = res_table.H_goal_strc + res_table.A_goal_strc
    res_table['goal_bilans'] = res_table.goal_zdob - res_table.goal_strc
    res_table['pts'] = res_table.H_pts + res_table.A_pts

    res_table['strzaly_oddane'] = res_table.H_strzaly_oddane + res_table.A_strzaly_oddane
    res_table['strzaly_dopuszczone'] = res_table.H_strzaly_dopuszczone + res_table.A_strzaly_dopuszczone
    res_table['strz_cel_oddane'] = res_table.H_strz_cel_oddane + res_table.A_strz_cel_oddane
    res_table['strz_cel_dopuszczone'] = res_table.H_strz_cel_dopuszczone + res_table.A_strz_cel_dopuszczone

    res_table['cor_wyk'] = res_table.H_kornery_wyk + res_table.A_kornery_wyk
    res_table['cor_bro'] = res_table.H_kornery_bro + res_table.A_kornery_bro

    # Sort by points - we create an imitation of the table
    res_table.sort_values(by=['pts','goal_bilans','goal_zdob'], inplace=True, ascending=(False,False,False))
    # NOTE: In the case of an equal number of points, the position in the table depends on the rules by specific league.
    # We will always assume the order: pts'> 'goal_bilans'>' goal_zdob '


    # # Number of matches played at home and away
    res_table['H_nmatch'] = results.groupby('HomeTeam').size()
    res_table['A_nmatch'] = results.groupby('AwayTeam').size()
    res_table['n_match'] = res_table['H_nmatch'] + res_table['A_nmatch']

    # Create characteristics
    res_table['pts_per_math'] = res_table.pts/res_table.n_match
    res_table['gz'] = res_table.goal_zdob/res_table.n_match
    res_table['gs'] = res_table.goal_strc/res_table.n_match
    res_table['sh_od'] = res_table.strzaly_oddane/res_table.n_match
    res_table['sh_ot'] = res_table.strzaly_dopuszczone/res_table.n_match
    res_table['cw'] = res_table.cor_wyk/res_table.n_match
    res_table['cb'] = res_table.cor_bro/res_table.n_match
    res_table['pozycja'] = range(1,len(res_table)+1)
    res_table

    ###########################################################################
    # The (almost) output table - data with created variables
    ###########################################################################

    output = pd.concat([form_var,
                        res_table[['pts_per_math','gz','gs','sh_od','sh_ot','cw','cb','pozycja']]
                       ], axis=1)
    
    h_var = output.loc[[HOME_TEAM] , : ]
    h_var.columns = ['h_'+i for i in h_var.columns]
    h_var.index = [0]

    a_var = output.loc[[AWAY_TEAM] , : ]
    a_var.columns = ['a_'+i for i in a_var.columns]
    a_var.index = [0]

    return pd.concat([match_res, h_var, a_var], axis=1)


In [2]:
# As argument pass max possible number
calc_features(378)

,Date,HomeTeam,AwayTeam,FTR,h_pts_3,h_pts_5,h_pts_7,h_gz_3,h_gz_5,h_gz_7,...,a_rc_5,a_rc_7,a_pts_per_math,a_gz,a_gs,a_sh_od,a_sh_ot,a_cw,a_cb,a_pozycja
0,2018-08-17,Girona,Valladolid,D,0.0,0.6,0.428571,0.666667,0.6,0.571429,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Save data to .csv - variables for each pair of teams in specific match
calc_features(378).to_csv('hist_data.csv')

for i in range(0,377):
    calc_features(377-i).to_csv('hist_data.csv', mode='a', header=False)